In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from eli5.sklearn import PermutationImportance
from sklearn.tree import DecisionTreeRegressor
from catboost import CatBoostRegressor,Pool
import matplotlib.patches as patch
from scipy.stats import kurtosis
import matplotlib.pyplot as plt
from sklearn.svm import NuSVR
from scipy.stats import skew
from scipy.stats import norm
from scipy import linalg
from sklearn import tree
from sklearn import svm
import lightgbm as lgb
import xgboost as xgb
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
import graphviz
import warnings
import random
import eli5
import shap  # package used to calculate Shap values
import time
import glob
import sys
import os

In [ ]:
%matplotlib inline
%precision 4
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
pd.set_option("display.precision", 15)

In [ ]:
%%time
train = pd.read_csv('../input/train.csv' , dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})

In [ ]:
print("Train has: rows:{} cols:{}".format(train.shape[0], train.shape[1]))

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')
submission.head()

In [ ]:
print("submission has: rows:{} cols:{}".format(submission.shape[0], submission.shape[1]))

In [ ]:
len(os.listdir(os.path.join('../input/', 'test')))

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.shape

In [ ]:
0 in train.values

In [ ]:
# let's select just 150000 rows

mini_train= pd.read_csv("../input/train.csv",nrows=150000)

In [ ]:
mini_train.describe()

In [ ]:
# check for missing values

mini_train.isna()

In [ ]:
mini_train.describe()

In [ ]:
mini_train.info()

In [ ]:
# basic histogram

mini_train.hist();

In [ ]:
# indvidual histograms

mini_train["acoustic_data"].hist();

In [ ]:
plt.plot(mini_train["time_to_failure"], mini_train["acoustic_data"])
plt.title("time_to_failure histogram")

In [ ]:
sns.distplot(mini_train.acoustic_data.values, color="Red", bins=100, kde=False)

In [ ]:
# kde plot
sns.kdeplot(mini_train["acoustic_data"] )

In [ ]:
'''Because we have only one feature(acoustic_data), and the size of the training 
set is very large( more that 60000000 rows), it is a good idea to reduce the size of the training set 
with making new segment and also to increase the number of attributes by using statistical attributes.'''

# based on : https://www.kaggle.com/inversion/basic-feature-benchmark
rows = 150000
segments = int(np.floor(train.shape[0] / rows))
segments

In [ ]:
# breaking the train dataset into 2 parts X_train and Y-train 

X_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['ave', 'std', 'max', 'min','sum','skew','kurt'])
y_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['time_to_failure'])

In [ ]:
# we created the data frames without any actual values
X_train.head()
# x_train is completely modified based on statistical values

**Think about this time to failure value**

In [ ]:
# https://tqdm.github.io/
# tqdm is used to show progress

for segment in tqdm(range(segments)):
    seg = train.iloc[segment*rows:segment*rows+rows]
    x = seg['acoustic_data'].values
    y_idx = seg['acoustic_data'].values.argmax()
    y = seg['time_to_failure'].values[y_idx]
    #print("time_to_failure for segment "+ str(segment)+"is:",y)
    
    
    y_train.loc[segment, 'time_to_failure'] = y
    X_train.loc[segment, 'ave'] = x.mean()
    X_train.loc[segment, 'std'] = x.std()
    X_train.loc[segment, 'max'] = x.max()
    X_train.loc[segment, 'min'] = x.min()
    X_train.loc[segment, 'sum'] = x.sum()
    X_train.loc[segment, 'skew'] =skew(x)
    X_train.loc[segment, 'kurt'] = kurtosis(x)

In [ ]:
# modified X_train

X_train.head()

In [ ]:
# modified Y_train

y_train.head()

In [ ]:
# creating test data for submission
X_test = pd.DataFrame(columns=X_train.columns, dtype=np.float64, index=submission.index)

In [ ]:
X_test.head()

In [ ]:
%%time
for seg_id in  X_test.index:
#     print('seg_id is '+seg_id)
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
#     print('seg_id is '+seg_id)
#     print(seg.shape)
#     print(seg.head())
    
    x = seg['acoustic_data'].values
    
    X_test.loc[seg_id, 'ave'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()
    X_test.loc[seg_id, 'sum'] = x.sum()
    X_test.loc[seg_id, 'skew'] =skew(x)
    X_test.loc[seg_id, 'kurt'] = kurtosis(x)

In [ ]:
X_test.shape

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [ ]:
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_scaled

In [ ]:
X_test_scaled

## Prediction models

### SVM

In [ ]:
svm = NuSVR()
svm.fit(X_train_scaled, y_train.values.flatten())
y_pred_svm = svm.predict(X_train_scaled)

In [ ]:
score = mean_absolute_error(y_train.values.flatten(), y_pred_svm)

In [ ]:
score

### Catboost

In [ ]:
train_pool = Pool(X,y)
cat_model = CatBoostRegressor(
                               iterations=3000,# change 25 to 3000 to get best performance 
                               learning_rate=0.03,
                               eval_metric='MAE',
                              )
cat_model.fit(X,y,silent=True)


In [ ]:
y_pred_cat = cat_model.predict(X_train_scaled)

In [ ]:
cat_score = mean_absolute_error(y_train.values.flatten(), y_pred_cat)

In [ ]:
cat_score

In [ ]:
y_pred_cat.shape

In [ ]:
y_train.values.flatten().shape

### Random forest

In [ ]:
rfc_model = RandomForestRegressor(random_state=0).fit(X_train_scaled, y_train.values.flatten())

In [ ]:
y_pred_rf = rfc_model.predict(X_train_scaled)

In [ ]:
rf_score = mean_absolute_error(y_train.values.flatten(), y_pred_rf)

In [ ]:
rf_score

In [ ]:
y_pred_rf

In [ ]:
y_pred_rf_sub=rfc_model.predict(X_test_scaled)

In [ ]:
y_pred_rf_sub

In [ ]:
submission['time_to_failure'] = y_pred_rf_sub
submission.to_csv('submission_rf.csv')